In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# %matplotlib widget
import seaborn as sns
import plotly.express as px

import folium
from folium.plugins import HeatMap

from wordcloud import WordCloud
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from scipy.stats import probplot
import pylab

import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

ModuleNotFoundError: No module named 'folium'

# 0. Import Data

## 0.1 Datasets Path

In [ ]:
# For local
# train_data_path = './dataset/train.csv'
# test_data_path = './dataset/test.csv'
# coe_price_data_path = './dataset/auxiliary-data/sg-coe-prices.csv'
# mrt_exist_data_path = './dataset/auxiliary-data/sg-mrt-existing-stations.csv'
# mrt_planned_data_path = './dataset/auxiliary-data/sg-mrt-planned-stations.csv'
# primary_school_data_path = './dataset/auxiliary-data/sg-primary-schools.csv'
# mall_data_path = './dataset/auxiliary-data/sg-shopping-malls.csv'
# stock_price_data_path = './dataset/auxiliary-data/sg-stock-prices.csv'

# For online
train_data_path = '/kaggle/input/cs5228-team31-dataset/train.csv'
test_data_path = '/kaggle/input/cs5228-team31-dataset/test.csv'
# submission_rent_path = '/kaggle/input/cs5228-team31-dataset/example-submission.csv'
coe_price_data_path = '/kaggle/input/cs5228-team31-dataset/sg-coe-prices.csv'
mrt_exist_data_path = '/kaggle/input/cs5228-team31-dataset/sg-mrt-existing-stations.csv'
mrt_planned_data_path = '/kaggle/input/cs5228-team31-dataset/sg-mrt-planned-stations.csv'
primary_school_data_path = '/kaggle/input/cs5228-team31-dataset/sg-primary-schools.csv'
mall_data_path = '/kaggle/input/cs5228-team31-dataset/sg-shopping-malls.csv'
stock_price_data_path = '/kaggle/input/cs5228-team31-dataset/sg-stock-prices.csv'

## 0.2 Load Data

In [ ]:
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)
# test_rent_col = pd.read_csv(submission_rent_path)
coe_price_data = pd.read_csv(coe_price_data_path)
mrt_exist_data = pd.read_csv(mrt_exist_data_path)
mrt_planned_data = pd.read_csv(mrt_planned_data_path)
primary_school_data = pd.read_csv(primary_school_data_path)
mall_data = pd.read_csv(mall_data_path)
stock_price_data = pd.read_csv(stock_price_data_path)

In [ ]:
print(train_data.shape, test_data.shape)

# 1. Cleaning

In [ ]:
train_data['flat_type'] = train_data['flat_type'].str.replace("-", " ")
train_data.head()

# 2. Encoding

## 2.0 Encoding Functions
The encoding functions we have now are: one-hot encoding, binary encoding(TODO), word embedding(TODO), normalization, dict mapping, function mapping, drop attribute.

In [ ]:
def encode_one_hot(data: pd.DataFrame, column_name: str):
    data = pd.get_dummies(data, columns=column_name)

def encode_binary(data: pd.DataFrame, column_name: str):
    categories = data[column_name].unique()
    num_bits = len(categories).bit_length()
    binary_encoding = {category: format(i, '0{0}b'.format(num_bits)) for i, category in enumerate(categories)}
    data[column_name] = data[column_name].map(binary_encoding)

def encode_word_embedding(data: pd.DataFrame, column_name: str):
    # TODO
    pass

def encode_normalization(data: pd.DataFrame, column_name: str):
    min_value = data[column_name].min()
    max_value = data[column_name].max()

    data[column_name] = (data[column_name] - min_value) / (max_value - min_value)

def encode_dict_map(data: pd.DataFrame, column_name: str, transform_map: dict):
    data[column_name] = data[column_name].map(transform_map)

def encode_function_map(data: pd.DataFrame, column_name: str, transform_func):
    data[column_name] = data[column_name].apply(transform_func)

def encode_drop(data: pd.DataFrame, column_name: str):
    data.drop(columns=[column_name], inplace=True)

## 2.1 Attribute: rent_approval_date
There are two ways to encode this attribute. First is function mapping, according to the function "year*10+month". Second is drop the attribute, since this attribute may be replaced by "lease commence date" or the stock amount. For now, I use the first way.

TODO: Consider the second way and see if there will be any improvement or not.

In [ ]:
def transform_func_date(value):
    splits = value.split('-')
    return int(splits[0]) * 100 + int(splits[1])

encode_function_map(train_data, 'rent_approval_date', transform_func_date)
encode_function_map(test_data, 'rent_approval_date', transform_func_date)
# encode_drop(train_data, 'rent_approval_date')
train_data.head()
test_data.head()

## 2.2 Attribute: town
There are four ways to encode this attribute. First is one-hot encoding. But there are 26 categories in this atribute. Second is binary encoding. Third is word embedding. Forth is drop this attribute, since this attribute may be replaced by "subzone-planning area-region" or "latitude-longitude". For now, I choose the second way.

TODO: Consider the forth way.

In [ ]:
# encode_one_hot(train_data, 'town')
encode_binary(train_data, 'town')
encode_binary(test_data, 'town')
# encode_word_embedding(train_data, 'town')
# encode_drop(train_data, 'town')
train_data.head()
test_data.head()

## 2.3 Attribute: block
There are two ways to encode. Since there are a lot of catigories in this attribute, I don't think it is a good idea to use one-hot encoding. And, since the value is not words, I don't think it have any semantic info and can be encoded by word embedding. So, first way is binary encoding. Second way is drop attribute, similar to attribute "town". For now, I choose the forth way.

TODO: Consider the second way.

In [ ]:
# encode_binary(train_data, 'block')
encode_drop(train_data, 'block')
encode_drop(test_data, 'block')
train_data.head()

## 2.4 Attribute: street name
There are three ways to encode. Similar to attribute "block", this attribute is not suitable for one-hot encoding. So, first way is binary encoding. Second way is word embedding, since the value in this attribute have semantic info. Third way is drop attribute, similar to attribute "town". For now, I choose the forth way.

TODO: Consider the forth way.

In [ ]:
# encode_binary(train_data, 'street_name')
# encode_word_embedding(train_data, 'street_name')
encode_drop(train_data, 'street_name')
encode_drop(test_data, 'street_name')
train_data.head()
test_data.head()

## 2.5 Attribute: flat type
There are three ways to encode. First is one-hot encoding since only 5 catagories in this attribute. Second is binary encoding. Third is dict mapping to map the type into number in order. For now, I choose the third way.

TODO: Change the map value

In [ ]:
# encode_one_hot(train_data, 'flat_type')
# encode_binary(train_data, 'flat_type')
flat_type_mapping_dict = {'2 room': 2, '3 room': 3, '4 room': 4, '5 room': 5, 'executive': 6,
                         '2-room': 2, '3-room': 3, '4-room': 4, '5-room': 5}
encode_dict_map(train_data, 'flat_type', flat_type_mapping_dict)
encode_dict_map(test_data, 'flat_type', flat_type_mapping_dict)
train_data.head()
test_data.head()

## 2.6 Attribute: flat model
There are four ways to encode. First is one-hot encoding. But there are XX catagories in this attribute. Second is bianry encoding. Third is word embedding. Forth is drop attribute. For now, I choose the second way.

In [ ]:
# encode_one_hot(train_data, 'flat_model')
encode_binary(train_data, 'flat_model')
encode_binary(test_data, 'flat_model')
# encode_word_embedding(train_data, 'flat_model')
# encode_drop(train_data, 'flat_model')
train_data.head()
test_data.head()

## 2.7 Attribute: floor area sqm
The value in this attribute is number. So we can use it directly or use it after normalization. For now, I use the normalization.

In [ ]:
encode_normalization(train_data, 'floor_area_sqm')
encode_normalization(test_data, 'floor_area_sqm')
train_data.head()
test_data.head()

## 2.7 Attribute: furnished
There are two way to encode. First is dict mapping. It map "yes" to True and "no" to False. However, since the value of this attribute for all data is "yes", the second way is drop attribute. For now, I choose the second way.

In [ ]:
# furnished_mapping_dict = {'no': False, 'yes': True}
# encode_dict_map(train_data, 'furnished', {'no': False, 'yes': True})
encode_drop(train_data, 'furnished')
encode_drop(test_data, 'furnished')
train_data.head()
test_data.head()

## 2.8 Attribute: lease commence date
Similar to attribute "rent_approval_date". There are two ways to encode this attribute. First is use directly. Second is drop the attribute, since this attribute may be replaced by "rent_approval_date" or the stock amount. For now, I use the first way.

TODO: Consider the second way and see if there will be any improvement or not.

In [ ]:
# encode_drop(train_data, 'lease_commence_date')
train_data.head()

## 2.9 Attribute: latitude & longitude
The value in these attributes are number. So we can use them directly or use them after normalization. In addition, we can also consider to drop these attributes and replace them by the distance to public facilities. For now, I choose normalization and keep these attributes.

In [ ]:
encode_normalization(train_data, 'latitude')
encode_normalization(train_data, 'longitude')

encode_normalization(test_data, 'latitude')
encode_normalization(test_data, 'longitude')
train_data.head()
test_data.head()

## 2.10 Attribute: elevation
Similar to attribute "furnished". For now, I decide to drop this attribute.

In [ ]:
encode_drop(train_data, 'elevation')
encode_drop(test_data, 'elevation')
train_data.head()
test_data.head()

## 2.11 Attribute: subzone
There are three ways to encode. Since there are a lot of catigories in this attribute, I don't think it is a good idea to use one-hot encoding. So, first way is binary encoding. Second way is word embedding. Third way is drop attribute, since this attribute may be replaced by "town-block-street_name" or "latitude-longitude". For now, I choose the third way.

TODO: Consider the third way.

In [ ]:
# encode_binary(train_data, 'subzone')
# encode_word_embedding(train_data, 'subzone')
encode_drop(train_data, 'subzone')
encode_drop(test_data, 'subzone')
train_data.head()
test_data.head()

## 2.12 Attribute: planning area
There are four ways to encode. First way is one-hot encoding. But there are 29 catagories in this attribute. Second is binary encoding. Third way is word embedding. Forth way is drop attribute, similar to attribute "subzone". For now, I choose the second way.

TODO: Consider the forth way.

In [ ]:
# encode_one_hot(train_data, 'planning_area')
encode_binary(train_data, 'planning_area')
encode_binary(test_data, 'planning_area')
# encode_word_embedding(train_data, 'planning_area')
# encode_drop(train_data, 'planning_area')
train_data.head()
test_data.head()

## 2.13 Attribute: region
There are three ways to encode. First is one-hot encoding since there are only 5 catagories in this attribute. Second is binary encoding. Third is drop attribute, similar to attribute "subzone". For now, I choose second way.

TODO: Consider the third way.

In [ ]:
# encode_one_hot(train_data, 'region')
encode_binary(train_data, 'region')
encode_binary(test_data, 'region')
# encode_drop(train_data, 'region')
train_data.head()
test_data.head()

## 2.14 Attribute: monthly rent
The value in this attribute is number. So we can use it directly or use it after normalization. For now, I use the normalization.

In [ ]:
# encode_normalization(train_data, 'monthly_rent')
# train_data.head()
# # test_data.head()

# 3. EDA - Main Data

**General Info**

In [ ]:
train_data.info()

**Descriptive Statistics**

In [ ]:
train_data.describe()

**Visualization of General Info**

In [ ]:
sns.pairplot(train_data,height=2)
plt.show()

Observations
* There is no null value for any column in the dataset.
* The scale of numerical data varies significantly, remember to normalize before analysis
* There are natural groups but not clear, further observation required

**Univariate Analysis**

Categorical - town & subarea

In [ ]:
# print(train_data['town'].unique().size, train_data['planning_area'].unique().size)

In [ ]:
# town_counts = train_data['town'].value_counts()
# planning_area_counts = train_data['planning_area'].value_counts()



# # Merge the two Series into a single DataFrame
# combined_df = pd.DataFrame({'Town': town_counts, 'Planning Area': planning_area_counts}).fillna(0)

# # Plotting
# plt.figure(figsize=(20, 10))

# # Width of a bar
# width = 0.4

# # Positions of bars on x axis
# r1 = range(len(combined_df))
# r2 = [x + width for x in r1]

# # Creating bars
# plt.bar(r1, combined_df['Town'], width=width, label='Town', color='blue', edgecolor='grey')
# plt.bar(r2, combined_df['Planning Area'], width=width, label='Planning Area', color='red', edgecolor='grey')

# # Title & Subtitle
# plt.title('Comparing Town and Planning Area Counts', fontweight='bold')

# # X axis
# plt.xlabel('Areas', fontweight='bold')
# plt.xticks([r + width for r in range(len(combined_df))], combined_df.index, rotation=90)

# # Y axis
# plt.ylabel('Counts', fontweight='bold')

# # Show the legend
# plt.legend()

# # Display the plot
# plt.tight_layout()
# plt.show()


Categorical - flat_type

In [ ]:
train_data['flat_type'].value_counts()

In [ ]:
sns.set_style('whitegrid')
fig,axes = plt.subplots(figsize=(6,4))
colors = ['#87ace8','#e3784d', '#6ecc64','#b644e3','#eb7c87']

ax = sns.countplot(x='flat_type',data=train_data, palette=['#e3784d','#87ace8', '#6ecc64','#b644e3','#eb7c87'])
for container in ax.containers:
    ax.bar_label(container)
plt.title('Frequency of different number of flat_type present in Houses available for Rent',fontsize=15)
plt.show()

Categorical - flat_model

In [ ]:
# train_data['flat_model'].value_counts()

In [ ]:
# flat_model_counts = train_data['flat_model'].value_counts()

# # Plotting
# plt.figure(figsize=(15,6))  # Adjust the figure size
# sns.barplot(y=flat_model_counts.index, x=flat_model_counts.values, palette='viridis')  # Use y for town names to get a horizontal bar plot

# # Add titles and labels
# plt.title('Number of Occurrences by Flat Model')
# plt.xlabel('Number of Occurrences')
# plt.ylabel('Flat Model')

# # Display the plot
# plt.tight_layout()
# plt.show()

Categorical - furnished

In [ ]:
# train_data['furnished'].value_counts()

Categorical - latitude & longitude

In [ ]:
# latitude = train_data['latitude']
# longitude = train_data['longitude']

# avg_lat, avg_lon = latitude.mean(), longitude.mean()

# # Create a base map
# m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

# # Add the heat map
# heat_data = [[lat, lon] for lat, lon in zip(train_data['latitude'], train_data['longitude'])]
# HeatMap(heat_data).add_to(m)

# # Display the map
# m

Categorical - subzone

In [ ]:
# train_data['subzone'].value_counts()

Categorical - region

In [ ]:
# train_data['region'].value_counts()

**Observation**
* town & planning_area: these two features' value overlaps, and there are 3 extra values in planning_area. **binary encoded** with other geographical features?
* flat_type: 4/3/5 rooms are most popular, does that mean sufficient **demand or supply**? (may influence price)
* furnished - 100% furnished, can we drop this feature?
* latitude & longitude - there are actually **naturally formed groups**, how to group them properly?
* subzone - maybe can be used with the latitude * longitude?
* region - 5 disctince values, correspond to the heat map

Numerical - monthly_rent

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 12))
ax1 = sns.histplot(x = train_data['monthly_rent'], color='teal', ax= ax[0, 0])
ax2 = sns.boxplot(x = train_data['monthly_rent'], ax= ax[0, 1], color= 'teal')
ax3 = sns.violinplot(x = train_data['monthly_rent'], ax= ax[1, 0], color= 'teal')
ax4 = probplot(train_data['monthly_rent'], plot=pylab)
pylab.show()

In [ ]:
# sns.set_context('notebook', font_scale = 1.3)
# plt.figure(figsize=(15, 4))
# ax = sns.barplot(x=train_data['region'], 
#                  y=train_data['monthly_rent'], 
#                  palette='viridis',
#                  ci = None)
# plt.ylabel('Monthly Rent');

# for p in ax.patches:
#     ax.annotate(int(p.get_height()), (p.get_x() + 0.4, p.get_height() + 1), ha = 'center', va = 'bottom', color = 'Black')

In [ ]:
sns.set_context('notebook', font_scale = 1.3)
plt.figure(figsize=(15, 4))
ax = sns.barplot(x=train_data['flat_type'], 
                 y=train_data['monthly_rent'], 
                 palette='viridis',
                 ci = None)
plt.ylabel('Monthly Rent');

for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x() + 0.4, p.get_height() + 1), ha = 'center', va = 'bottom', color = 'Black')

In [ ]:
# sns.set_context('notebook', font_scale = 1.3)
# plt.figure(figsize=(15, 4))
# ax = sns.barplot(x=train_data['flat_model'], 
#                  y=train_data['monthly_rent'], 
#                  palette='viridis',
#                  ci = None)
# ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
# plt.ylabel('Monthly Rent');

# for p in ax.patches:
#     ax.annotate(int(p.get_height()), (p.get_x() + 0.4, p.get_height() + 1), ha = 'center', va = 'bottom', color = 'Black')

In [ ]:
train_data = train_data.sort_values('lease_commence_date')

# Plot
plt.figure(figsize=(16, 4))
sns.lineplot(x='lease_commence_date', y='monthly_rent', data=train_data, ci=None)

plt.title('Trend of Monthly Rent Over Years')
plt.xlabel('Year of Lease Commencement')
plt.ylabel('Average Monthly Rent')
plt.tight_layout()
plt.show()


Numerical - floor_area_sqm

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))

sns.histplot(train_data['floor_area_sqm'], bins=30, kde=False, color='orange', ax=ax[0])  # Adjust position as desired
sns.scatterplot(x=train_data['floor_area_sqm'], y=train_data['monthly_rent'], ax=ax[1], color='green', alpha=0.5)  # Adjust position as desired

plt.tight_layout()
plt.show()

Monthly rent in the map

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111, projection='3d')

x = train_data['latitude']
y = train_data['longitude']
z = train_data['monthly_rent']

scatter = ax.scatter(x, y, z, c=z, marker='o', alpha=0.6, cmap="viridis")
ax.set_xlabel('Latitude', labelpad=10) 
ax.set_ylabel('Longitude', labelpad=10) 
ax.set_zlabel('Monthly Rent', labelpad=10)
fig.colorbar(scatter, ax=ax, label='Monthly Rent', pad=0.1, shrink=0.5) 

ax.view_init(elev=60, azim=-60)  # Adjust elevation & azimuthal angles as needed

plt.tight_layout()
plt.show()

In [ ]:
# train_data['town'].value_counts()

In [ ]:
import plotly.express as px

fig = px.scatter_3d(train_data, 
                    x='latitude', 
                    y='longitude', 
                    z='monthly_rent',
                    color='monthly_rent', 
                    opacity=0.6,
                    color_continuous_scale='viridis',
                    title="3D Scatter plot of Rent based on Latitude and Longitude")

fig.show()

**Observation**
1. monthly rent
* it's not heavily skewed, but there are definately outliers 
* regions - the average rent across different regions are roughly the same
* flat_type - the average rent is positively correlated to the #room
* flat_model - those with extremely low frequency in flat_model usually have a high price 
* lease_commence_date - graph indicates the rent is increasing yearly, and exhibit up and downs in each decade. this can be combined with the auxiliary data to analyze the influence of economics
* floor_area_sqm - generally normal distribution, but obvious outliers
* latitude&longitude - observe from a higher angle, there are definately groups

* log-2023-9-29：
1. 这个b-platform太难用了，可以考虑代替；
2. 对于二进制数据，无法作为模型的输入，需要改成int or float，同时onehot编码可以考虑以其他方式应用；
3. 需要尝试更多方法进行整合，但当前需要规范好input。

# 4. Modelling (by sxg)

Different modelling methods may involve different data processing and feature engineering techniques. Therefore, instead of using the train_data/test_data directly, we create separate copies of the dataset for each method to generate the corresponding training/testing data for that particular method.

## 4.1 Multiple Linear Regression

In [ ]:
# train_data_mlr = train_data.copy()

In [ ]:
# X_train = train_data_mlr.drop(['monthly_rent'], axis = 1)
# y_train = train_data_mlr['monthly_rent']
# # X_test = test_data_mlr.drop(['monthly_rent'], axis = 1)
# # y_test = test_data_mlr['monthly_rent']

### Model Training

In [ ]:
# from sklearn.linear_model import LinearRegression

In [ ]:
# # Instantiate a Linear Regression model, for simplicity here we first take all attributes into consideration
# mlr = LinearRegression()
# mlr.fit(X_train, y_train)

### Evaluation

#### Evaluate the model on training dataset

We first use metrics to evaluate the model performance on the training dataset.

In [ ]:
# y_pred_train = mlr.predict(X_train)

In [ ]:
# # Model evaluation metrics
# print('R^2:',metrics.r2_score(y_train, y_pred_train))
# print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
# print('MAE:',metrics.mean_absolute_error(y_train, y_pred_train))
# print('MSE:',metrics.mean_squared_error(y_train, y_pred_train))
# print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

In [ ]:
# # Visualizing
# plt.scatter(y_train, y_pred_train)
# plt.xlabel("Actual Price")
# plt.ylabel("Predicted Price")
# plt.title("Actual vs Predicted")
# plt.show()

**Assumptions on MLR Models**

There are some main assumptions on Multiple Regression Models, which are listed below:

* Linearity
* No Multi-Collinearity
* Homoskedasticity
* Independence of independent variable
* Normality
* Independence of errors

We should conduct examinations on whether the assumptions above are satisfied in this context. Basic methods include using diagnostic plots for the relevant analysis. For example, we can use residual plots to check if homoskedasticity is violated or the errors' independency. Also, scatter plots of dependent versus independent variables can help us find non-linear relationships and so on.

* **Linearity**

Linearity in MLR means that the relationship between the independent variables (features) and the dependent variable (target) is linear.

In [ ]:
# X_train.columns

In [ ]:
# numerical_attributes = ['floor_area_sqm', 'latitude', 'longitude']
# X_plot = X_train.loc[:,numerical_attributes]
# fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(12, 4))

# axs = axs.flatten()
# for i, k in enumerate(numerical_attributes):
#     sns.regplot(y=y_train, x=X_plot[k], ax=axs[i], scatter_kws={"color": "teal"}, line_kws={"color": "red"})
# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

* **No Multi-Collinearity**

Multicollinearity occurs when two or more independent variables in the MLR model are highly correlated, making it difficult to separate their individual effects on the dependent variable. It can impact the reliability of your regression model's coefficients and predictions. 

We use the Correlation Matrix to have a look at the multi-collinearity across all variables first. Look for high correlation coefficients (typically greater than 0.7) between pairs of variables. High correlations suggest multicollinearity.

In [ ]:
# # Display Correlation Matrix
# corrMatrix = X_train.corr()
# sns.heatmap(corrMatrix, annot=True)
# plt.show()

`flat_type` and `floor_area_sqm` has a correlation coefficient of 0.95, and `town` and `planning_area` has a correlation coefficient of 0.97. This may suggest multicollinearity.

Note that `flat_type`, `town` and `planning_area` are  encoded and is not numeric values by nature, so we may consider other encoding methods for avoiding multi-collinearity across the variables.

Then we use another metrics Variance Inflation Factor (VIF) to measure how much the variance of the estimated regression coefficients is increased due to multi-collinearity. Calculate the VIF for each independent variable. High VIF values (typically greater than 10) indicate multicollinearity.

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# def calc_vif(X):
#     vif = pd.DataFrame()
#     vif["variables"] = X.columns
#     vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
#     return(vif)

In [ ]:
# calc_vif(X_train)
## DOES NOT WORK 
## Ensure that the input data you're using with the isfinite function is of a numeric data type (e.g., float, int) because isfinite is typically used to check the finiteness of numeric values. If your data contains non-numeric or incompatible data types, you'll encounter this error.

* **Homoskedasticity**

Homoscedasticity means that the variance of the residuals (the differences between the observed values and the predicted values) is constant across all levels of the independent variables. If homoscedasticity is violated, it can lead to problems such as biased coefficient estimates and incorrect inference.

In [ ]:
# plt.scatter(y_pred_train, y_train - y_pred_train)
# plt.title("Residual vs Fit Plot")
# plt.xlabel("Predicted")
# plt.ylabel("Residuals")
# plt.show()

We can observe that there is a random scattering of points with no discernible pattern. As in a homoskedastic dataset, the points should be evenly scattered around the horizontal line at 0 (the residuals have constant variance), we can come to the conclusion that the Homoskedasticity is not violated.

Also, we do not observe a funnel-shaped, fan-shaped pattern, or any other systematic change in the spread of residuals as the fitted values change, which suggests heteroskedasticity (the variance of residuals is not constant).

* **Independence of independent variables**

MLR assumes that independent variables are not perfectly correlated with each other (i.e., they are independent). Violations of this assumption can lead to unstable coefficient estimates and difficulties in interpreting their individual effects.

In [ ]:
# # Independence of X variables-'floor_area_sqm'
# plt.scatter(X_train['floor_area_sqm'], y_train - y_pred_train)
# plt.title("Floor Area Size vs Residuals")
# plt.xlabel("floor_area_sqm")
# plt.ylabel("Residuals")
# plt.show()

In [ ]:
# # Independence of X variables-'latitude'
# plt.scatter(X_train['latitude'], y_train - y_pred_train)
# plt.title("Latitude vs Residuals")
# plt.xlabel("latitude")
# plt.ylabel("Residuals")
# plt.show()

In [ ]:
# # Independence of X variables-'longitude'
# plt.scatter(X_train['longitude'], y_train - y_pred_train)
# plt.title("Longitude vs Residuals")
# plt.xlabel("latitude")
# plt.ylabel("Residuals")
# plt.show()

The plots above all show that residuals do appear randomly and symmetrically distributed around zero under all conditions, which prove the independence.

* **Normality**

MLR assumes that the residuals (the differences between the observed and predicted values) are normally distributed. Deviations from normality can impact the validity of statistical inference, such as hypothesis tests and confidence intervals.

In [ ]:
# # Normality of residuals
# sns.distplot(y_train - y_pred_train)
# plt.title("Normality of Residuals")
# plt.xlabel("Residuals")
# plt.ylabel("Frequency")
# plt.show()

We can see fro the plot above that the residuals are normally distributed, which means the normality assumption is satiefied.

* **Independence of errors**

Independence of errors states that the residuals (the differences between the observed values and the predicted values) should be independent of each other. Violations of this assumption can lead to incorrect parameter estimates, unreliable hypothesis tests, and inaccurate predictions. 

Here we use *Durbin-Watson Test* for checking independence of errors. This statistical test checks for the presence of autocorrelation in the residuals. A Durbin-Watson statistic value close to 2 indicates no autocorrelation.

In [ ]:
# durbin_watson_statistic = sm.stats.stattools.durbin_watson(y_train - y_pred_train)
# print(f'Durbin-Watson Statistic: {durbin_watson_statistic}')

The Durbin-Watson statistic value is close to 2, which means there is no autocorrelation (i.e. the assmuption is satisfied).

#### Evaluate the model on test dataset

After the evalution on the training dataset, as well as the checks for MLR assmuptions and corresponding model refinement, we can use the model as the final  MLR model and evaluate its performance on the test dataset.

In [ ]:
# y_pred_test = mlr.predict(X_test)

In [ ]:
# mlr_dict = {'Model':'Multiple Linear Regression',
#           'R^2':metrics.r2_score(y_test, y_pred_test),
#           'Adjusted R^2':(1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)),
#           'MAE':metrics.mean_absolute_error(y_test, y_pred_test),
#           'MSE':metrics.mean_squared_error(y_test, y_pred_test),
#           'RMSE':np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))}

# mlr_metrics = pd.DataFrame.from_dict(mlr_dict, orient = 'index').T

# # Display model performance metrics
# mlr_metrics

### Analysis

> TODO

</br>

## 4.2 Random Forest Regressor

In [ ]:
# train_data_rf = train_data.copy()
# # test_data_rf = test_data.copy()

In [ ]:
# X_train = train_data_rf.drop(['monthly_rent'], axis = 1)
# y_train = train_data_rf['monthly_rent']
# # X_test = test_data_mlr.drop(['monthly_rent'], axis = 1)
# # y_test = test_data_mlr['monthly_rent']

### Model Training

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from sklearn.ensemble import RandomForestRegressor

We use GridSearchCV as a technique to systematically search through different combinations of hyperparameters to find the best set of hyperparameters for RF model.

In [ ]:
# rf_param_grid = {
#     'max_depth': [90, 100], # Maximum number of levels in each decision tree
# #     'max_depth': [80, 90, 100],
#     'max_features': [2, 3], # Maximum number of features considered for splitting a node
#     'min_samples_leaf': [1, 3], # Minimum number of data points allowed in a leaf node
# #     'min_samples_leaf': [1, 3, 4, 5],
#     'n_estimators': [300, 600] # Number of trees in the forest
# #     'n_estimators': [100, 300, 600]
# }

In [ ]:
# rf_reg = RandomForestRegressor(random_state = 42)
# rf_grid = GridSearchCV(estimator = rf_reg, param_grid = rf_param_grid, cv=5, n_jobs=-1, verbose=2)

In [ ]:
# rf_grid.fit(X_train, y_train)

In [ ]:
# rf_grid.best_params_

In [ ]:
# # Get best RF estimator based on best parameters gemerated from above
# rf = rf_grid.best_estimator_

### Evaluation

#### Evaluate the model on training dataset

We first use metrics to evaluate the model performance on the training dataset.

In [ ]:
# # Use RF regressor to predict on training set
# y_pred_train = rf.predict(X_train)

In [ ]:
# # Model evaluation metrics
# print('R^2:',metrics.r2_score(y_train, y_pred_train))
# print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
# print('MAE:',metrics.mean_absolute_error(y_train, y_pred_train))
# print('MSE:',metrics.mean_squared_error(y_train, y_pred_train))
# print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

#### Evaluate the model on testing dataset

In [ ]:
# y_pred_test = rf.predict(X_test)

In [ ]:
# rf_dict = {'Model':'Random Forest Regressor',
#           'R^2':metrics.r2_score(y_test, y_pred_test),
#           'Adjusted R^2':(1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)),
#           'MAE':metrics.mean_absolute_error(y_test, y_pred_test),
#           'MSE':metrics.mean_squared_error(y_test, y_pred_test),
#           'RMSE':np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))}

# rf_metrics = pd.DataFrame.from_dict(rf_dict, orient = 'index').T

# # Display model performance metrics
# rf_metrics

### Analysis

* **Importance of the features**

In [ ]:
# rf.feature_importances_

In [ ]:
# rf_features = pd.DataFrame([X_train.columns, rf.feature_importances_]).T
# rf_features = rf_features.rename(columns={0: 'Feature', 1: 'Importance Score'})
# rf_features.sort_values(by = 'Importance Score', ascending = False)

</br>

## 4.3 XGBoost Regressor

In [ ]:
# train_data_rf = train_data.copy()
# # test_data_rf = test_data.copy()

In [ ]:
# X_train = train_data_rf.drop(['monthly_rent'], axis = 1)
# y_train = train_data_rf['monthly_rent']
# # X_test = test_data_mlr.drop(['monthly_rent'], axis = 1)
# # y_test = test_data_mlr['monthly_rent']

### Model Training

In [ ]:
# import xgboost as xgb
# from xgboost import plot_importance

In [ ]:
# xgb_param_grid = {"learning_rate": [0.05, 0.1, 0.2], # Step size shrinkage used in update to prevents overfitting.
#                   "max_depth"        : [6, 8, 9, 10], # Maximum depth of a tree. 
#                   "min_child_weight" : [1, 3, 5, 7], # Minimum number of instances required in a child node
#                   "gamma"            : [0.0, 0.1, 0.2, 0.3], # Minimum loss reduction required to make a further partition on a leaf node of the tree.
#                   "colsample_bytree" : [0.3, 0.4, 0.6, 0.8] # Number of features supplied to a tree
# }

In [ ]:
# xgb_reg = xgb.XGBRegressor(seed = 42, objective ='reg:squarederror')
# xgb_grid = GridSearchCV(estimator = xgb_reg, param_grid = xgb_param_grid, cv=5, n_jobs=-1, verbose=2)

**TO BE CHECKED!!! Failed**

In [ ]:
# xgb_grid.fit(X_train, y_train)

In [ ]:
# xgb_grid.best_params_

In [ ]:
# Get best XGBoost estimator based on best parameters gemerated from above
# xgb = xgb_grid.best_estimator_

### Evaluation

#### Evaluate the model on training dataset

We first use metrics to evaluate the model performance on the training dataset.

In [ ]:
# # Use XGBoost regressor to predict on training set
# y_pred_train = xgb.predict(X_train)

In [ ]:
# # Model evaluation metrics
# print('R^2:',metrics.r2_score(y_train, y_pred_train))
# print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
# print('MAE:',metrics.mean_absolute_error(y_train, y_pred_train))
# print('MSE:',metrics.mean_squared_error(y_train, y_pred_train))
# print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)))

#### Evaluate the model on testing dataset

In [ ]:
# y_pred_test = xgb.predict(X_test)

In [ ]:
# xgb_dict = {'Model':'XGBoost Regressor',
#           'R^2':metrics.r2_score(y_test, y_pred_test),
#           'Adjusted R^2':(1 - (1-metrics.r2_score(y_test, y_pred_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)),
#           'MAE':metrics.mean_absolute_error(y_test, y_pred_test),
#           'MSE':metrics.mean_squared_error(y_test, y_pred_test),
#           'RMSE':np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))}

# xgb_metrics = pd.DataFrame.from_dict(xgb_dict, orient = 'index').T

# # Display model performance metrics
# xgb_metrics

### Analysis

* **Importance of the features**

In [ ]:
# xgb_features = pd.DataFrame([X_train.columns, xgb.feature_importances_]).T
# xgb_features = xgb_features.rename(columns={0: 'Feature', 1: 'Importance Score'})
# xgb_features.sort_values(by = 'Importance Score', ascending = False).head(15)

In [ ]:
# plot_importance(xgb, max_num_features = 10)

</br>

## 4.4 LightGBM Regressor

In [ ]:
# train_data_rf = train_data.copy()
# # test_data_rf = test_data.copy()

In [ ]:
# X_train = train_data_rf.drop(['monthly_rent'], axis = 1)
# y_train = train_data_rf['monthly_rent']
# # X_test = test_data_mlr.drop(['monthly_rent'], axis = 1)
# # y_test = test_data_mlr['monthly_rent']

#### Model Training

In [ ]:
# import lightgbm as lgb

In [ ]:
# gbm_param_grid = {'metric': ['rmse'],
#                   'max_depth': [9,10,11,12,13], 
#                   'bagging_fraction': [0.8, 0.9, 1],
#                   'feature_fraction': [0.8, 0.9, 1],
#                   'min_data_in_leaf': [20,50,80],
#                   'learning_rate': [0.01,0.05,0.1,0.2]}

In [ ]:
# lgb_reg = lgb.LGBMRegressor(seed = 42, num_iterations = 1200)

# lgb_grid = GridSearchCV(estimator = lgb_reg, param_grid = gbm_param_grid, cv=5, n_jobs=-1, verbose=1)

**TO BE CHECKED**

In [ ]:
#lgb_grid.fit(X_train, y_train)

## 5. Model Comparison

In [ ]:
# df_metrics = pd.concat([mlr_metrics, rf_metrics, xgb_metrics, lgbm_metrics])
# df_metrics.sort_values(by = 'RMSE', ascending = True)

See from the metrics comparison above, XXXXXXX achieves the best performance on the test set with an RMSE of XXXXXXX. 

# 5. Ensemble Models

5.1 Baseline

In [ ]:
pip install d2l

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import ParameterGrid, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV, Lasso, ElasticNet, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
import xgboost as xgb
import lightgbm as lgb
from mlxtend.regressor import StackingCVRegressor

In [ ]:
# *******************Linear model*******************
def get_model():
    net = nn.Sequential(nn.Linear(in_features, 64),
                        nn.ReLU(),
                        nn.Linear(64, 32),
                        nn.ReLU(),
                        nn.Linear(32, 1),)
    return net

def log_rmse(net, features, labels):
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # Adam optimization
    optimizer = torch.optim.Adam(net.parameters(),
                                 lr = learning_rate,
                                 weight_decay = weight_decay)
    
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay,
           batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_model()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            d2l.plot(list(range(1, num_epochs + 1)), [train_ls, valid_ls],
                     xlabel='epoch', ylabel='rmse', xlim=[1, num_epochs],
                     legend=['train', 'valid'], yscale='log')
        print(f'fold {i + 1}, train log rmse {float(train_ls[-1]):f}, '
              f'valid log rmse {float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

def train_and_pred(train_features, test_features, train_labels, test_data,
                   num_epochs, lr, weight_decay, batch_size):
    net = get_model()
    train_ls, _ = train(net, train_features, train_labels, None, None,
                        num_epochs, lr, weight_decay, batch_size)
    d2l.plot(np.arange(1, num_epochs + 1), [train_ls], xlabel='epoch',
             ylabel='log rmse', xlim=[1, num_epochs], yscale='log')
    print(f'train log rmse {float(train_ls[-1]):f}')
    preds = net(test_features).detach().numpy()
    df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
    test_data['Predicted'] = pd.Series(preds.reshape(1, -1)[0])
    submission = pd.concat([df_id['Id'], test_data['Predicted']], axis=1)
    return submission, train_ls

In [ ]:
train_data.reset_index(drop=True, inplace=True)
train_data.head()

In [ ]:
train_len = train_data.shape[0] * 0.8
numeric_attributes=['flat_type', 'floor_area_sqm', 'latitude', 'longitude']
train_data_linear = train_data.loc[:train_len-1,numeric_attributes]
train_data_rent = train_data.loc[:train_len-1,['monthly_rent']]
train_data_rent.shape

In [ ]:
test_data_linear = train_data.loc[train_len:,numeric_attributes]
test_data_linear.head()
print(train_data_linear.shape, test_data_linear.shape)

In [ ]:
train_attributes = torch.tensor(train_data_linear.values, dtype=torch.float32)
test_attributes = torch.tensor(test_data_linear.values, dtype=torch.float32)
train_labels_linear = torch.tensor(train_data_rent.values.reshape(-1, 1), dtype=torch.float32)
loss = nn.MSELoss()
in_features = train_attributes.shape[1]

In [ ]:
# train linear model
k, num_epochs, lr, weight_decay, batch_size = 5, 600, 0.01, 0, 64 # adjust hyperparameter
train_l, valid_l = k_fold(k, train_attributes, train_labels_linear, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-fold cross-validation: average train log rmse: {float(train_l):f}, '
      f'average test log rmse: {float(valid_l):f}')

In [ ]:
# predict linear model
submission, train_ls = train_and_pred(train_attributes, test_attributes, train_labels_linear, test_data,
               num_epochs, lr, weight_decay, batch_size)
# score = self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_linear_rmsle'+str(float(train_ls[-1]))[:6]+'.csv', index=False)

In [ ]:
# Define error metrics
def rmse(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

# k-fold Cross Validation
# set verbose=3 for more details
def rmsle_cross_val(X_train, y_train, n_folds, model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return rmse

kfolds = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# *******************Lasso model*******************
# lasso_alphas = [0.00001, 0.00005, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
# model_lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1000,alphas =lasso_alphas, random_state=42, cv=kfolds))
# model_lasso = make_pipeline(RobustScaler(), Lasso(max_iter=1000, alpha =0.00005, random_state=42))
model_lasso = Lasso(max_iter=1000, alpha =0.00005, tol=0.1, random_state=42)
score_lasso = rmsle_cross_val(train_data_linear, train_data_rent, 5, model_lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score_lasso.mean(), score_lasso.std()))
model_lasso.fit(train_data_linear.values, train_data_rent.values)
model_lasso_pred = model_lasso.predict(test_data_linear.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_lasso_pred), columns = ['Predicted'])
model_lasso_pred, df_pred['Predicted']
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_lasso_rmse'+str(float(score_lasso.mean()))[:7]+'.csv', index=False)

In [ ]:
# *******************ENet model*******************
# enet_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
# enet_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
model_ENet = make_pipeline(RobustScaler(), ElasticNet(max_iter=3000, alpha=0.0005, l1_ratio=0.8, tol=0.2, random_state=42))
score_ENet = rmsle_cross_val(train_data_linear, train_data_rent, 5, model_ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score_ENet.mean(), score_ENet.std()))
model_ENet.fit(train_data_linear.values, train_data_rent.values)
model_ENet_pred = model_ENet.predict(test_data_linear.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_ENet_pred), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_enet_rmse'+str(float(score_ENet.mean()))[:7]+'.csv', index=False)

In [ ]:
# *******************Ridge model*******************
ridge_alphas = [1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
model_ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kfolds))
# model_ridge = make_pipeline(RobustScaler(), Ridge(max_iter=1000, alpha=0.1))
score_ridge = rmsle_cross_val(train_data_linear, train_data_rent, 5, model_ridge)
print("Ridge score: {:.4f} ({:.4f})\n".format(score_ridge.mean(), score_ridge.std()))
model_ridge.fit(train_data_linear.values, train_data_rent.values)
model_ridge_pred = model_ENet.predict(test_data_linear.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_ridge_pred), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_ridge_rmse'+str(float(score_ridge.mean()))[:7]+'.csv', index=False)

In [ ]:
# *******************SVR model*******************
# C = [0.1, 0.5, 1, 50, 100, 1000]
# epsilon = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10]
# C:5 - 0.1163 (0.0032), C:10 - 0.1140 (0.0034), epsilon: 0.05 - 0.1105 (0.0039)
model_svr = make_pipeline(RobustScaler(), SVR(C=10, epsilon=0.05, gamma='scale'))
score_svr = rmsle_cross_val(train_data_linear, np.log(train_data_rent), 5, model_svr)
print("SVR score: {:.4f} ({:.4f})\n".format(score_svr.mean(), score_svr.std()))

model_svr.fit(train_data_linear.values, np.log(train_data_rent.values))
model_svr_pred = model_svr.predict(test_data_linear.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(np.exp(model_svr_pred)), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_svr_rmsle'+str(float(score_svr.mean()))[:7]+'.csv', index=False)

In [ ]:
test_data.head()

In [ ]:
train_data.head()

In [ ]:
# init train data and test data for non-linear model
all_data_ensemble = train_data.copy()
# test_data_linear = train_data.loc[train_len*0.8:,numeric_attributes]
train_data_ensemble = train_data.copy().drop(columns = ['monthly_rent'])
test_data_ensemble = test_data.copy().drop(columns = ['Predicted'])
train_data_rent = train_data.loc[:train_len/0.8,['monthly_rent']]
train_data_ensemble.shape, test_data_ensemble.shape

In [ ]:
# *******************Random Forest model*******************
# use grid search to produce a baseline rf model
model_rf = RandomForestRegressor(oob_score=True, random_state=3, n_jobs=-1)
params ={
    'n_estimators': [1300],# [800, 900, 1000, 1200, 1500],
    'min_samples_leaf': [1],# [1, 2, 3, 5, 10, 25],
    'max_features': [0.5],# [None, 0.5, 'sqrt', 'log2'],
    'max_depth': [35],# [5, 6, 7, 8, 10, 15, 20],
    'min_samples_split': [2]# [2, 3, 4]
}

best_score = 0
for g in ParameterGrid(params):
    model_rf.set_params(**g)
    model_rf.fit(train_data_ensemble, train_data_rent)
    if model_rf.oob_score_ > best_score:
        best_score = model_rf.oob_score_
        best_grid = g
        print('oob:', best_score, best_grid)
# oob: 0.9764868974915827 {'max_depth': 35, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1300}

In [ ]:
# check feature importance
def RF_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}).sort_values('imp', ascending=False)

model_rf = RandomForestRegressor(n_jobs=-1, n_estimators=1300, oob_score=True, max_depth=35, min_samples_leaf=1, min_samples_split=2, max_features=0.5)
model_rf.fit(train_data_ensemble, train_data_rent)
feature_imp = RF_importance(model_rf, train_data_ensemble)
print(model_rf.oob_score_)
print(feature_imp)

In [ ]:
# feature selection
Threshold = 0.0009
to_keep = feature_imp[feature_imp.imp>Threshold].cols
to_keep = [col for _, col in to_keep.items()]
print(to_keep)
df_keep = train_data_ensemble[to_keep].copy()
model_rf = RandomForestRegressor(n_jobs=-1, n_estimators=1300, oob_score=True, max_depth=35, min_samples_leaf=1, min_samples_split=2, max_features=0.5)
model_rf.fit(df_keep, train_data_rent)
score_rf = rmsle_cross_val(train_data_ensemble, train_data_rent, 5, model_rf)
print(model_rf.oob_score_, score_rf.mean())

In [ ]:
# deep selection
# keep_cols imp >= 0.01
keep_cols = ['rent_approval_date', 'latitude', 'longitude', 'floor_area_sqm', 'flat_type', 'lease_commence_date', 'flat_model', 'planning_area', 'town', 'region']
cols = to_keep
scores = []
feats = []
for col in cols:
    tmp = to_keep.copy()
    if col in keep_cols:
        continue
    tmp.remove(col)
    df_tmp = train_data_ensemble[tmp].copy()
    model_rf = RandomForestRegressor(n_jobs=-1, n_estimators=1300, oob_score=True, max_depth=35, min_samples_leaf=1, min_samples_split=2, max_features=0.5)
    model_rf.fit(df_tmp, train_data_rent)
    scores.append(model_rf.oob_score_)
    feats.append(col)

to_del = sorted(zip(scores, feats), reverse=True)
to_del

In [ ]:
# fine-tuning
# delete increase less than 0.0001
to_keep_final = ['latitude', 'lease_commence_date', 'longitude', 'flat_type', 'flat_model', 'floor_area_sqm', 
                 'rent_approval_date']
train_data_final = train_data_ensemble[to_keep_final].copy()
model_rf = RandomForestRegressor(oob_score=True, n_jobs=-1)
params ={
    'n_estimators': [1200, 1250,1280],# [1200, 1250, 1280, 1300, 1350, 1400],
    'min_samples_leaf': [1],# [1, 2, 3, 5, 10, 25],
    'max_features': [0.5],# [0.5, 'sqrt', 'log2'],
    'max_depth': [35],# [5, 6, 7, 8],
    'min_samples_split': [2]# [2, 3, 4]
}

best_score = 0
for g in ParameterGrid(params):
    model_rf.set_params(**g)
    model_rf.fit(train_data_final, train_data_rent)
    if model_rf.oob_score_ > best_score:
        score_rf = rmsle_cross_val(train_data_final, train_data_rent, 5, model_rf)
        best_score = model_rf.oob_score_
        best_grid = g
        print('best oob:', best_score, best_grid, 'score:', score_rf.mean())

In [ ]:
model_rf = RandomForestRegressor(n_jobs=-1, n_estimators=1250, max_depth=35, min_samples_leaf=1, min_samples_split=2, max_features=0.5)
model_rf.fit(train_data_final, train_data_rent)

In [ ]:
test_data_final = test_data_ensemble[to_keep_final]
# print(test_data_final)
model_rf_pred=model_rf.predict(test_data_final)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_rf_pred), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_rf_'+str(best_score)[:7]+'.csv', index=False)

In [ ]:
# *******************GBoost model*******************
model_GBoost = GradientBoostingRegressor(n_estimators=7000, learning_rate=0.1,
                                   max_depth=5, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
score_GBoost = rmsle_cross_val(train_data_ensemble, np.log(train_data_rent), 5, model_GBoost)
print("GradientBoostingRegressor score: {:.4f} ({:.4f})\n".format(score_GBoost.mean(), score_GBoost.std()))

model_GBoost.fit(train_data_ensemble.values, train_data_rent.values)
model_gb_pred = model_GBoost.predict(test_data_ensemble.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_gb_pred), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_gb_rmsle'+str(float(score_GBoost.mean()))[:7]+'.csv', index=False)

In [ ]:
# *******************XGB model*******************
# xgb_params = dict(
#     max_depth=6,           # maximum depth of each tree - try 2 to 10
#     learning_rate=0.01,    # effect of each tree - try 0.0001 to 0.1
#     n_estimators=1000,     # number of trees (that is, boosting rounds) - try 1000 to 8000
#     min_child_weight=1,    # minimum number of houses in a leaf - try 1 to 10
#     colsample_bytree=0.7,  # fraction of features (columns) per tree - try 0.2 to 1.0
#     subsample=0.7,         # fraction of instances (rows) per tree - try 0.2 to 1.0
#     reg_alpha=0.5,         # L1 regularization (like LASSO) - try 0.0 to 10.0
#     reg_lambda=1.0,        # L2 regularization (like Ridge) - try 0.0 to 10.0
#     num_parallel_tree=1,   # set > 1 for boosted random forests
# )
model_xgb = xgb.XGBRegressor(learning_rate=0.01, gamma=0,
                             max_depth=5, min_child_weight = 1,
                             n_estimators=6000, colsample_bytree=0.6,
                             reg_alpha=0.5, reg_lambda=1.0,
                             subsample=0.7, random_state = 42, nthread = -1)
score_xgb = rmsle_cross_val(train_data_ensemble, np.log(train_data_rent), 5, model_xgb)
print("XGB score: {:.4f} ({:.4f})\n".format(score_xgb.mean(), score_xgb.std()))

model_xgb.fit(train_data_ensemble.values, train_data_rent.values)
model_xgb_pred = model_xgb.predict(test_data_ensemble.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_xgb_pred), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_xgb_rmsle'+str(float(score_xgb.mean()))[:7]+'.csv', index=False)

In [ ]:
# *******************LightGBM model*******************
# Use large max_bin (may be slower)
# Use small learning_rate with large num_iterations
# Use large num_leaves (may cause over-fitting)
model_lgb = lgb.LGBMRegressor(objective='regression', 
                       num_leaves=25, learning_rate=0.05, 
                       max_bin = 200, n_estimators=1000,
                       random_state=42)
score_lgb = rmsle_cross_val(train_data_ensemble, np.log(train_data_rent), 5, model_lgb)
print("LGB score: {:.4f} ({:.4f})\n".format(score_lgb.mean(), score_lgb.std()))

model_lgb.fit(train_data_ensemble.values, train_data_rent.values)
model_lgb_pred = model_lgb.predict(test_data_ensemble.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_lgb_pred), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_lgb_rmsle'+str(float(score_lgb.mean()))[:7]+'.csv', index=False)

In [ ]:
# *******************RF model*******************
model_RF = RandomForestRegressor(n_jobs=-1, n_estimators=1250, oob_score=True, max_depth=35, min_samples_leaf=1, min_samples_split=2, max_features=0.5)
model_RF.fit(train_data_ensemble.values, train_data_rent.values)

In [ ]:
# *******************Stacking model*******************
model_stack = StackingCVRegressor(regressors=(model_GBoost, model_xgb, model_lgb),
                                meta_regressor=model_rf,
                                use_features_in_secondary=True)
score_stack = rmsle_cross_val(train_data_ensemble, np.log(train_data_rent), 5, model_stack)
print("Stacked score: {:.4f} ({:.4f})\n".format(score_stack.mean(), score_stack.std()))
# fitting stacked model
model_stack.fit(train_data_ensemble.values, train_data_rent.values)
model_stack_pred = model_stack.predict(test_data_ensemble.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_stack_pred), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_stacked_rmsle'+str(float(score_stack.mean()))[:7]+'.csv', index=False)

In [ ]:
# *******************Blend models/Weighting*******************
def blended_predictions(X):
    return ((0.05 * model_GBoost.predict(X)) + \
            (0.1 * model_xgb.predict(X)) + \
            (0.1 * model_lgb.predict(X)) + \
            (0.55 * model_RF.predict(X)) + \
            (0.2 * model_stack.predict(X)))

model_blend_pred = blended_predictions(train_data_ensemble.values)
score_blend = rmse(np.log(train_data_rent.values), np.log(model_blend_pred))
print(score_blend)

In [ ]:
model_blended_pred = blended_predictions(test_data_ensemble.values)
df_id = pd.DataFrame([id for id in range(test_data.shape[0])], columns=['Id'])
df_pred = pd.DataFrame(pd.Series(model_blended_pred), columns = ['Predicted'])
submission = pd.concat([df_id['Id'], df_pred['Predicted']], axis=1)
# self_evaluation(submission)
submission.to_csv('/kaggle/working/submission_blended_rmsle'+str(score_blend)[:7]+'.csv', index=False)